In [0]:
#Google drive authentication Code
!pip install -U -q PyDrive
!pip install gensim
import numpy as np
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# 2. Load a file by ID and create local file.
#(**********************Steps to Get File ID******************************)
#Upload the dataset on google drive and then go and click on the dataset to scroll to the option which says get shareable link

#The link will be copied to the clipboard paste it to the notepad and then
#like : https://drive.google.com/open?id=1DHkNolpOVuE76ZL2bTHY_LDwIC126XFn
#in this link copy the the string after id = and paste it in the code down here to get your data exported in the google collaboratory environment

##****************************************************************************************************************************
#Loading Book 1 of Game of thrones
downloaded = drive.CreateFile({'id':'1DHkNolpOVuE76ZL2bTHY_LDwIC126XFn'}) # replace fileid with Id of file you want to access represented by long string in single quotes
downloaded.GetContentFile('got1.txt') # now you can use export.csv share
# Loading book2 of Game of thrones
downloaded1 = drive.CreateFile({'id':'1McCXKaSp4OB5qZs-wowk3jjEZP7E7GWi'}) # replace fileid with Id of file you want to access represented by long string in single quotes
downloaded1.GetContentFile('got2.txt') # now you can use export.csv share
#loading book3 of Game of thrones
downloaded2 = drive.CreateFile({'id':'16XBPdsLk1-1RKLdeJHsnoTuvS5tWDo3p'}) # replace fileid with Id of file you want to access represented by long string in single quotes
downloaded2.GetContentFile('got3.txt') # now you can use export.csv share
# Loading book4 of Game of thrones
downloaded3 = drive.CreateFile({'id':'1sRX8_vKTRGt7tUFYP7iqIF6VVazLYXiU'}) # replace fileid with Id of file you want to access represented by long string in single quotes
downloaded3.GetContentFile('got4.txt') # now you can use export.csv share
#loading book5 of Game of thrones
downloaded4 = drive.CreateFile({'id':'1ekOnx6aSEWvHs9B9BXYBf_G5-y0LM58O'}) # replace fileid with Id of file you want to access represented by long string in single quotes
downloaded4.GetContentFile('got5.txt') # now you can use export.csv share


In [2]:
#Aim is to create word vectors from Game of the thrones dataset and analyse semantic similarity

#importing future as it is the bridge between the syntax of python3 and python2
from __future__ import absolute_import, division, print_function

#using library" codecs" to encode the word
import codecs
#importing regex to file searching using regular expressions
#For concurrency importing multiprocessing to enable multithreading so that each thread performs some sort of process
import multiprocessing
import os
# for printing human readable texts importing pprint
import pprint
#for more granular regex string processing we are going to import glob
import glob
import re
#natural language tool kit 

#IT has ability to produce token given text
#It has a pretrained model and has prerecorded lexicons
import nltk


#word2vec: created by Google to train words as huge collection of vectors

#Using sklearn.manifold for dimensionality reduction
import sklearn.manifold

import numpy as  np
import matplotlib.pyplot as plt
#importing pandas framework


import pandas as pd
#visualising the dataset using seaborn



import gensim.models.word2vec as w2v


    100% |████████████████████████████████| 15.9MB 75kB/s 
  Using cached smart_open-1.5.6.tar.gz
  Using cached boto3-1.5.22-py2.py3-none-any.whl
  Using cached boto-2.48.0-py2.py3-none-any.whl
  Using cached bz2file-0.98.tar.gz
  Using cached jmespath-0.9.3-py2.py3-none-any.whl
  Using cached s3transfer-0.1.12-py2.py3-none-any.whl
  Using cached botocore-1.8.36-py2.py3-none-any.whl
    100% |████████████████████████████████| 552kB 1.9MB/s 
  Running setup.py bdist_wheel for smart-open ... - done
  Stored in directory: /content/.cache/pip/wheels/36/48/35/97efc2bd1b233627131c9a936c9de23681846db707b907d353
  Running setup.py bdist_wheel for bz2file ... - done
  Stored in directory: /content/.cache/pip/wheels/31/9c/20/996d65ca104cbca940b1b053299b68459391c01c774d073126
Successfully built smart-open bz2file


In [3]:
#Cleaning the data
#Step 1: Preprocessing

#punkt and stopwords are two funcitons on nltk
#punkt is a tokeniser
#punkt takes a sentence and produces the keywords as token
nltk.download('punkt')

#stopwords removes and, like, the, of, at
nltk.download('stopwords')





[nltk_data] Downloading package punkt to /content/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /content/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
#Step 2
#parsing the good files
book_filenames = sorted(glob.glob('*.txt'))
book_filenames

['got1.txt', 'got2.txt', 'got3.txt', 'got4.txt', 'got5.txt']

In [5]:
#Combining all these books into one string
#initialising a raw corpus in unicode format for this
corpus_raw = u""
for book_filename in book_filenames:
    print("Reading '{0}'.....".format(book_filename))
    with codecs.open(book_filename, "r","utf-8") as book_file:
         corpus_raw += book_file.read()
    print("Corpus  is {0} characters long".format(len(corpus_raw)))


Reading 'got1.txt'.....
Corpus  is 1770659 characters long
Reading 'got2.txt'.....
Corpus  is 4071041 characters long
Reading 'got3.txt'.....
Corpus  is 6391405 characters long
Reading 'got4.txt'.....
Corpus  is 8107945 characters long
Reading 'got5.txt'.....
Corpus  is 9719485 characters long


In [0]:
#Splitting the corpus into sentences for further steps
#using nltk's punkt
#right now we want sentences as tokens from our corpus
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

#Applying the model to our fetched corpus
raw_sentences = tokenizer.tokenize(corpus_raw)





In [0]:
#defining the function which converts sentences to wordlist
#remove unneccesary words without hyphens
#wordlist is 
def sentence_to_wordlist(raw):
    clean = re.sub("[^a-zA-Z]"," ", raw)
    words = clean.split()
    return words

In [0]:
#once we have the list of words from above function we will take each of the words and tokenise it
#initialise a empty sentence list

sentences =[]
for raw_sentence in raw_sentences:
    if len(raw_sentence) > 0:
        sentences.append(sentence_to_wordlist(raw_sentence))

In [9]:
#Illustration with an example
#print an example
print(raw_sentences[4])
print(sentence_to_wordlist(raw_sentences[4]))

Maps by James Sinclair.
[u'Maps', u'by', u'James', u'Sinclair']


In [10]:
#Now getting total no of tokens in the corpus

token_count =- sum([len(sentence) for sentence in sentences])
print("The entire corpus contains {0:}  tokens".format(abs(token_count)))

The entire corpus contains 1818103  tokens


In [0]:
#Training a Word2Vec
#once we have the vectors it can be used for distance, similarity and ranking
#step 2 build our model, another one is Glove
#define hyperparameters

#num_features is the dimensionality of word vectors
#the more is the dimension more is the computation but it will be more generalised and accurate
num_features = 300


#A vector is a type of tensor in tensorflow
# Minimum word count threshold.
min_word_count = 3

# Number of threads to run in parallel.
num_workers = multiprocessing.cpu_count()

# Context window length, it is like looking at block of 7 words at an instance.
context_size = 7


# Downsample setting for frequent words.
#as per lit: the more frequent the word is the less we need to create its vector as it already has that
#It is taken to be between 0 and 1e-5
downsampling = 1e-3

# Seed for the random number generation
seed = 1

In [0]:
#word2vec model imported from gensim library

Suraj_word2vec = w2v.Word2Vec(
    sg=1,
    seed=seed,
    workers=num_workers,
    size=num_features,
    min_count=min_word_count,
    window=context_size,
    sample=downsampling,
    iter = 10
)

In [0]:
Suraj_word2vec.build_vocab(sentences)
#print("Word2Vec vocabulary length:", len(Suraj_word2vec.vocab))

In [25]:
#train model on sentneces
Suraj_word2vec.train(sentences,total_examples=Suraj_word2vec.corpus_count,epochs=Suraj_word2vec.iter, report_delay=1.0, compute_loss=True)
# getting the training loss value


14042762

In [0]:
#save model
if not os.path.exists("trained"):
    os.makedirs("trained")

In [0]:
Suraj_word2vec.save(os.path.join("trained", "thrones2vec.w2v"))

In [0]:
#load model
Suraj_word2vec = w2v.Word2Vec.load(os.path.join("trained", "thrones2vec.w2v"))

In [0]:
#squash dimensionality to 2
#https://www.oreilly.com/learning/an-illustrated-introduction-to-the-t-sne-algorithm
tsne = sklearn.manifold.TSNE(n_components=2, random_state=0)

In [0]:
#put it all into a giant matrix
#all_word_vectors_matrix = Suraj_word2vec.w2v.syn0

In [48]:
""""#plot point in 2d space
points = pd.DataFrame(
    [
        (word, coords[0], coords[1])
        for word, coords in [
            (word, all_word_vectors_matrix_2d[Suraj_word2vec.vocab[word].index])
            for word in Suraj_word2vec.vocab
        ]
    ],
    columns=["word", "x", "y"]
)
""""

SyntaxError: ignored

In [0]:
""""points.head(10)
sns.set_context("poster")
points.plot.scatter("x", "y", s=10, figsize=(20, 12))""""

In [46]:
""""def plot_region(x_bounds, y_bounds):
    slice = points[
        (x_bounds[0] <= points.x) &
        (points.x <= x_bounds[1]) & 
        (y_bounds[0] <= points.y) &
        (points.y <= y_bounds[1])
    ]
    
    ax = slice.plot.scatter("x", "y", s=35, figsize=(10, 8))
    for i, point in slice.iterrows():
        ax.text(point.x + 0.005, point.y + 0.005, point.word, fontsize=11)"""

'"def plot_region(x_bounds, y_bounds):\n    slice = points[\n        (x_bounds[0] <= points.x) &\n        (points.x <= x_bounds[1]) & \n        (y_bounds[0] <= points.y) &\n        (points.y <= y_bounds[1])\n    ]\n    \n    ax = slice.plot.scatter("x", "y", s=35, figsize=(10, 8))\n    for i, point in slice.iterrows():\n        ax.text(point.x + 0.005, point.y + 0.005, point.word, fontsize=11)'

In [40]:
#plot_region(x_bounds=(4.0, 4.2), y_bounds=(-0.5, -0.1))
#plot_region(x_bounds=(0, 1), y_bounds=(4, 4.5))
Suraj_word2vec.most_similar("Stark")


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[(u'Eddard', 0.6202936768531799),
 (u'executed', 0.4770504832267761),
 (u'Snowbeard', 0.4548262357711792),
 (u'Edrick', 0.4435146749019623),
 (u'SHIREI', 0.4421089291572571),
 (u'Winterfell', 0.4412658214569092),
 (u'fishwife', 0.43919655680656433),
 (u'absently', 0.4310927391052246),
 (u'LOTHOR', 0.4267766773700714),
 (u'Underfoot', 0.424993097782135)]

In [41]:
Suraj_word2vec.most_similar("Aerys")

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[(u'II', 0.5767824649810791),
 (u'Jaehaerys', 0.5020555257797241),
 (u'Defiance', 0.47191619873046875),
 (u'Conciliator', 0.46519413590431213),
 (u'Mad', 0.4619624614715576),
 (u'Maekar', 0.4515315294265747),
 (u'Robert', 0.44786351919174194),
 (u'V', 0.44561100006103516),
 (u'Arrogant', 0.4392484426498413),
 (u'Beggar', 0.43634605407714844)]

In [42]:
Suraj_word2vec.most_similar("Aerys")

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[(u'II', 0.5767824649810791),
 (u'Jaehaerys', 0.5020555257797241),
 (u'Defiance', 0.47191619873046875),
 (u'Conciliator', 0.46519413590431213),
 (u'Mad', 0.4619624614715576),
 (u'Maekar', 0.4515315294265747),
 (u'Robert', 0.44786351919174194),
 (u'V', 0.44561100006103516),
 (u'Arrogant', 0.4392484426498413),
 (u'Beggar', 0.43634605407714844)]

In [0]:
#distance, similarity, and ranking
def nearest_similarity_cosmul(start1, end1, end2):
    similarities = Suraj_word2vec.most_similar_cosmul(
        positive=[end2, start1],
        negative=[end1]
    )
    start2 = similarities[0][0]
    print("{start1} is related to {end1}, as {start2} is related to {end2}".format(**locals()))
    return start2

In [50]:
nearest_similarity_cosmul("Stark", "Winterfell", "Riverrun")
nearest_similarity_cosmul("Jaime", "sword", "wine")
nearest_similarity_cosmul("Arya", "Nymeria", "dragons")

Stark is related to Winterfell, as Tully is related to Riverrun
Jaime is related to sword, as strongwine is related to wine
Arya is related to Nymeria, as wyrms is related to dragons


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `most_similar_cosmul` (Method will be removed in 4.0.0, use self.wv.most_similar_cosmul() instead).
  after removing the cwd from sys.path.


u'wyrms'